# Apply Ranking Recovery Algorithms

In [1]:
import pandas as pd
import seaborn as sns

from fairpair import *

In [2]:
# create a FairPairGraph with disadvantaged minority
G = FairPairGraph()
G.generate_groups(15, 5)
G.group_assign_scores(nodes=G.majority_nodes, distr=Distributions.normal_distr)
G.group_assign_scores(nodes=G.minority_nodes, distr=Distributions.normal_distr, loc=0.3, scale=0.2) # give a disadvantage to the minority

In [3]:
# run 100 iterations of ProbKnockout sampling
sampler = OversampleMinority(G, warn=False)
sampler.apply(iter=1000, k=1)
# apply davidScore for ranking recovery
ranker = RankRecovery(G)
ranking, other_nodes = ranker.apply(rank_using=davidScore) # by default, apply rankCentrality method
ranker._print_with_score(ranking) # sorted by rank score

,node,orig score,rank score
0,6,0.808455,22.841510
1,14,0.621010,21.604426
2,13,0.512326,14.302932
3,2,0.613737,11.448713
4,0,0.616879,10.846605
5,5,0.862593,9.706722
6,7,0.541242,8.677644
7,9,0.546671,6.448091
8,8,0.269521,4.828662
9,4,0.496484,0.108165


## Metrics

In [4]:
# score-based metrics
print("Majority NDCG (by score):", NDCG(G.majority, ranking)) # NDCG is only calculated as a "in-group" comparison because it is normalized
print("Minority NDCG (by score):", NDCG(G.minority, ranking))
print("Majority MSE (by score):", MSE(G.majority, ranking))
print("Minority MSE (by score):", MSE(G.minority, ranking))

Majority NDCG (by score): 0.9721691036554142
Minority NDCG (by score): 1.0
Majority MSE (by score): 0.0537064709457218
Minority MSE (by score): 0.07933076475103859


In [5]:
# rank-based metrics
print("Majority mean error (by rank):", rank_mean_error(G, ranking, G.majority))
print("Minority mean error (by rank):", rank_mean_error(G, ranking, G.minority))
print("Majority MSE (by rank):", rank_MSE(G, ranking, G.majority))
print("Minority MSE (by rank):", rank_MSE(G, ranking, G.minority))
print("Majority Spearman's rho:", spearmanr(G, ranking, G.majority))
print("Minority Spearman's rho:", spearmanr(G, ranking, G.minority))

Majority mean error (by rank): -0.5
Minority mean error (by rank): 1.0
Majority MSE (by rank): 3.5
Minority MSE (by rank): 5.4
Majority Spearman's rho: PearsonRResult(statistic=0.9035714891476541, pvalue=0.00033623167191990447)
Minority Spearman's rho: PearsonRResult(statistic=0.9295172508680996, pvalue=0.02222350284454909)


In [6]:
# weighted Kendall tau
print("Majority weighted Kendall tau:", weighted_tau(G, ranking, G.majority))
print("Minority weighted Kendall tau:", weighted_tau(G, ranking, G.minority))

Majority weighted Kendall tau: 0.1697556492155082
Minority weighted Kendall tau: 0.15277946444783344


In [7]:
print("Overall Spearman r:", spearmanr(G, ranking))
print('--')
print("Overall weighted Kendall tau:", weighted_tau(G, ranking))

Overall Spearman r: PearsonRResult(statistic=0.8892857142857142, pvalue=9.192974758842205e-06)
--
Overall weighted Kendall tau: 0.16358761184285264
